Welcome to my Easy Wav2Lip colab!

My goal is to make lipsyncing with this tool easy, fast and great looking!

Please view the GitHub for instructions: [https://github.com/anothermartz/Easy-Wav2Lip](https://github.com/anothermartz/Easy-Wav2Lip?tab=readme-ov-file#best-practices)

In [1]:
version = 'v8.3'
#@title <h1>Step 1: Setup "Easy-Wav2Lip"</h1> With one button: it's really that easy!
#@markdown 👈 Click that little circle play button first - it will ask for Google Drive access: <br>
#@markdown > Accept if your files are on Google Drive (recommended).
#@markdown <br> Alternatively, you can click deny and upload files manually, but this is slower.

#check if already installed
import os
import sys
if os.path.exists('installed.txt'):
  with open('last_file.txt', 'r') as file:
    last_file = file.readline()
  if last_file == version:
    sys.exit('Easy-Wav2Lip '+version+' has already been run on this instance!')


#check GPU is enabled
print('checking for GPU')
import torch
if not torch.cuda.is_available():
  sys.exit('No GPU in runtime. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')

#prompt to mount google drive
print('requesting Google Drive access')
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("google drive not linked")


#start timer
import time
start_time = time.time()

#clone git
giturl = 'https://github.com/anothermartz/Easy-Wav2Lip.git'


!git clone -b {version} {giturl}
%cd 'Easy-Wav2Lip'
working_directory = os.getcwd()
!mkdir 'face_alignment' 'temp'

#install prerequisites
print('installing batch_face')
import warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        module='torchvision.transforms.functional_tensor')
!pip install batch_face --quiet
!pip install basicsr==1.4.2 --quiet
print('fixing basicsr degradations.py')
!cp /content/Easy-Wav2Lip/degradations.py /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py
print('installing gfpgan')
!pip install gfpgan --quiet

!python install.py

from IPython.display import clear_output
clear_output()
print("Installation complete, move to Step 2!")

#end timer
elapsed_time = time.time() - start_time
from easy_functions import format_time
print(f"Execution time: {format_time(elapsed_time)}")

Installation complete, move to Step 2!
Execution time: 49s


In [3]:
import os
import sys
import time
import ipywidgets as widgets
from IPython.display import display, HTML, Image, clear_output
from google.colab import files
import configparser

if not os.path.exists('installed.txt'):
  sys.exit('Step 1 has not been run in this instance! Please run step 1 each time you disconnect from a runtime.')

print("Starting Step 2: Upload input files")

############################## user inputs #####################################
#@markdown <h1>Step 2: Upload input files:</h1>

# Create upload widgets
video_upload = widgets.FileUpload(accept='.mp4,.avi,.mov', multiple=False, description='Upload Video')
audio_upload = widgets.FileUpload(accept='.wav,.mp3', multiple=False, description='Upload Audio (Optional)')

# Display upload widgets
display(HTML("<b>Upload Video File:</b>"))
display(video_upload)
display(HTML("<b>Upload Audio File (Optional):</b>"))
display(audio_upload)


# Create an output widget to display status messages
status_output = widgets.Output()
display(status_output)

# Function to save uploaded files
def save_uploaded_file(uploaded_file, file_type):
  print("Saving uploaded file. Please wait ...")
  if not uploaded_file:
      return ""
  file_name = next(iter(uploaded_file))
  content = uploaded_file[file_name]['content']
  path = f"/content/{file_name}"
  with open(path, "wb") as f:
      f.write(content)
  print("Saving uploaded file done.")
  return path

print("Step 2 setup completed. Please upload your files and click 'Validate and Process'.")

#@markdown # Quality
quality = "Enhanced" # @param ["Fast", "Improved", "Enhanced"]
#@markdown * <b><u>Fast</u></b>: Wav2Lip <br>
#@markdown * <b><u>Improved</u></b>: Wav2Lip with a feathered mask around the mouth to remove the square around the face <br>
#@markdown * <b><u>Enhanced</u></b>: Wav2Lip + mask + GFPGAN upscaling done on the face
output_height = "full resolution" #@param ["half resolution", "full resolution", "480"] {allow-input: true}
use_previous_tracking_data = True #@param {type:"boolean"}

#@markdown # [Advanced tweaking](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#advanced-tweaking) (optional)
wav2lip_version = "Wav2Lip_GAN" # @param ["Wav2Lip", "Wav2Lip_GAN"]
nosmooth = True #@param {type:"boolean"}
#@markdown ### [Padding:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#padding)</h1> (Up, Down, Left, Right) <br>
U = 0 #@param {type:"slider", min:-100, max:100, step:1}
D = 10 #@param {type:"slider", min:-100, max:100, step:1}
L = 0 #@param {type:"slider", min:-100, max:100, step:1}
R = 0 #@param {type:"slider", min:-100, max:100, step:1}

#@markdown ### [Mask:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)
size = 1.5 #@param {type:"slider", min:1, max:6, step:0.1}
feathering = 1 #@param {type:"slider", min:0, max:3, step:1}
mouth_tracking = True #@param {type:"boolean"}
debug_mask = False #@param {type:"boolean"}

#@markdown # [Other options:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)
batch_process = True #@param {type:"boolean"}
output_suffix = "_Easy-Wav2Lip" #@param {type:"string"}
include_settings_in_suffix = False #@param {type:"boolean"}
preview_input = False #@param {type:"boolean"}
preview_settings = False #@param {type:"boolean"}
frame_to_preview = 100 # @param {type:"integer"}


Starting Step 2: Upload input files


FileUpload(value={}, accept='.mp4,.avi,.mov', description='Upload Video')

FileUpload(value={}, accept='.wav,.mp3', description='Upload Audio (Optional)')

Output()

Step 2 setup completed. Please upload your files and click 'Validate and Process'.


In [ ]:
with status_output:
    clear_output()

    print("Starting file validation and processing")

    if not video_upload.value:
        raise Exception("Please upload a video file before validating.")

    print("Saving uploaded files...")
    video_file = save_uploaded_file(video_upload.value, "Video")
    vocal_file = save_uploaded_file(audio_upload.value, "Audio")

    print(f"Video file saved: {video_file}")
    if vocal_file:
        print(f"Audio file saved: {vocal_file}")
    else:
        print("No audio file uploaded.")

    print("\nStarting configuration...")

    # Create a ConfigParser object
    config = configparser.ConfigParser()

    # Put all your variables in a dictionary
    options = {
        'video_file': video_file,
        'vocal_file': vocal_file,
        'quality': quality,
        'output_height': output_height,
        'wav2lip_version': wav2lip_version,
        'use_previous_tracking_data': use_previous_tracking_data,
        'nosmooth': nosmooth
    }
    padding = {
        'U': U,
        'D': D,
        'L': L,
        'R': R
    }
    mask = {
        'size': size,
        'feathering': feathering,
        'mouth_tracking': mouth_tracking,
        'debug_mask': debug_mask
    }
    other = {
        'batch_process': batch_process,
        'output_suffix': output_suffix,
        'include_settings_in_suffix': include_settings_in_suffix,
        'preview_input': preview_input,
        'preview_settings': preview_settings,
        'frame_to_preview': frame_to_preview
    }

    # Add the dictionary to the ConfigParser object
    config['OPTIONS'] = options
    config['PADDING'] = padding
    config['MASK'] = mask
    config['OTHER'] = other

    # Write the data to an INI file
    with open('config.ini', 'w') as f:
        config.write(f)

    print("Configuration completed. Starting processing...")

    # Run the processing script
    !python run.py

    print("Processing completed. Displaying results...")

    # Display results
    from easy_functions import show_video
    from IPython.display import Image
    if preview_settings:
        if os.path.isfile(os.path.join('temp','preview.jpg')):
            display(Image(os.path.join('temp','preview.jpg')))
    else:
        if os.path.isfile(os.path.join('temp','output.mp4')):
            print(f"Loading video preview...")
            show_video(os.path.join('temp','output.mp4'))

    print("Results displayed. Initiating file download...")

    # Download the generated file
    output_file = os.path.join('temp','output.mp4')
    if os.path.isfile(output_file):
        files.download(output_file)
        print(f"File '{output_file}' has been downloaded.")
    else:
        print(f"Output file '{output_file}' not found. Please check the processing results.")

    print("Step 2 completed.")